In [1]:
# 通过openai包直接调用其他的大模型，例如此处的千问大模型
import os
from openai import OpenAI
client = OpenAI(
    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
    api_key=os.getenv("DASHSCOPE_API_KEY"), 
    # api_key='sk-25186dbe2f984cbcba48ffca2c81a78f',
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)
completion = client.chat.completions.create(
    model="qwen-plus", # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    messages=[
        {'role': 'system', 'content': 'You are a helpful assistant.'},
        {'role': 'user', 'content': '你是谁？'}],
    )
    
print(completion.model_dump_json())

{"id":"chatcmpl-536b5211-5e83-9680-a89a-caf254b0d3db","choices":[{"finish_reason":"stop","index":0,"logprobs":null,"message":{"content":"我是通义千问，由阿里云开发的AI助手。我可以回答各种问题、提供信息和与用户进行对话。有什么我可以帮助你的吗？","refusal":null,"role":"assistant","audio":null,"function_call":null,"tool_calls":null}}],"created":1729739359,"model":"qwen-plus","object":"chat.completion","service_tier":null,"system_fingerprint":null,"usage":{"completion_tokens":33,"prompt_tokens":22,"total_tokens":55,"completion_tokens_details":null,"prompt_tokens_details":null}}


In [18]:
# 通过langchain的langchain_openai包直接调用其他的大模型，例如此处的千问大模型
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(
    temperature=0.95,
    model="glm-4",
    openai_api_key=os.getenv("ZHIPU_API_KEY"),
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
)
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a nice chatbot having a conversation with a human."
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)
#交流缓存记忆，将沟通的信息进行存储
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)
conversation.invoke({"question": "tell me a joke"})




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: tell me a joke

> Finished chain.


{'question': 'tell me a joke',
 'chat_history': [HumanMessage(content='tell me a joke'),
  AIMessage(content="Why don't scientists trust atoms?\n\nBecause they make up everything! 😄")],
 'text': "Why don't scientists trust atoms?\n\nBecause they make up everything! 😄"}

In [34]:
#MessagesPlaceholder的应用
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder("history"),
    ("human", "{question}")
])

prompt.invoke({
    "history": [("human", "what's 5 + 2"), ("ai", "5 + 2 is 7")],
    "question": "now multiply that by 4"
})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant.'), HumanMessage(content="what's 5 + 2"), AIMessage(content='5 + 2 is 7'), HumanMessage(content='now multiply that by 4')])

In [37]:
#存储记忆的提取
memory.save_context({"input": "你好！"} ,{"outputs": "你好，有什么我可以帮助你的吗？"})
chat_history = memory.load_memory_variables({"input": "..."})
print(chat_history["chat_history"])

[HumanMessage(content='tell me a joke'), AIMessage(content="Why don't scientists trust atoms?\n\nBecause they make up everything! 😄"), HumanMessage(content='你好！'), AIMessage(content='你好，有什么我可以帮助你的吗？'), HumanMessage(content='你好！'), AIMessage(content='你好，有什么我可以帮助你的吗？')]


In [39]:
# 调用智谱大模型进行对话
from langchain_community.chat_models import ChatZhipuAI
zhipuai_chat = ChatZhipuAI(
        temperature=0.5,
        api_key=os.getenv("ZHIPU_API_KEY"),
        model="glm-4",
    )
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
messages = [
    AIMessage(content="Hi."),
    SystemMessage(content="Your role is a poet."),
    HumanMessage(content="三打白骨精题材."),
]
# 打印结果
response = zhipuai_chat.invoke(messages)
print(response.content)

《三打白骨精》是中国古典名著《西游记》中的一个著名章节，讲述了唐僧师徒四人在取经路上遇到善于变化的白骨精的故事。以下是一首以这个题材为基础的诗：

西行路险多妖氛，
白骨精藏巧机关。
三番变化迷人眼，
一试禅心见佛缘。

初化村姑施怜悯，
再成老妇演悲欢。
三作将军威风凛，
真假难辨雾中烟。

悟空火眼识妖怪，
金箍棒下不容宽。
护师取经历磨难，
骨碎精消证菩提。


In [44]:
# 通过agent 查询网站资料回答问题
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=2)]
prompt = hub.pull("hwchase17/react")

# Choose the LLM to use
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, handle_parsing_errors=True,verbose=True)
agent_executor.invoke({"input": "什么是钎料，国内市场占有率是多少?"})



> Entering new AgentExecutor chain...
这个问题涉及到专业术语和市场份额，我需要使用搜索引擎来获取相关信息。

Action: tavily_search_results_json
Action Input: 钎料是什么

Observation[{'url': 'https://www.zhihu.com/market/pub/120034989/manuscript/1289567400316968960', 'content': '知乎盐选 | 1.2\u3000钎料的种类及使用 1.2\u3000钎料的种类及使用 1.2\u3000钎料的种类及使用 ① 适当（低于母材）的熔点。 ② 具有良好的润湿性能及填缝性能。 ③ 能与母材发生充分作用并形成牢固的结合。 ④ 具有稳定和均匀的化学成分。 ⑤ 能满足使用要求。 钎料一般按以下三种方法分类：按钎料的熔点分类，通常将熔点在 450℃ 以下的钎料称为软钎料，而高于 450℃ 的称为硬钎料，高于 950℃ 的称为高温钎料；按钎料的化学成分分类，软钎料分为 Sn 基、Bi 基、Pb 基等，硬钎料可分为 Al 基、Ag 基、Cu 基、Mn 基和 Au 基等；按钎焊工艺性能分类，分为自钎性钎料、真空钎料、复合钎料等。 钎料 S-Sn60Pb39Sb1（HL600）是含锡 60％的共晶型锡铅钎料，其熔点低，流动性好，用于钎焊工作温度较低及要求钎缝光洁的零件，如无线电零件、电气开关零件、计算机零件、易熔金属制品以及热处理件的钎焊。 钎料 S-Sn18Pb80Sb2（HL601）是锡的质量分数为 18％的锡铅钎料，其含锡量很低，熔点高，结晶晶格大。因此当用烙铁进行钎焊时比较困难，力学性能也比较差。主要用于钎焊铜及铜合金和镀锌铁皮等强度要求不高的零件。 钎料 S-Sn30Pb68Sb2（HL602）锡的质量分数为 30％，是应用较广的锡铅钎料。其铺展性能及力学性能都较好。常用于钎焊铜、黄铜、铁、镀锌铁皮、白铁皮等，如散热器、仪表、无线电元件、电缆护套及电机的扎线等产品。 钎料 S-Sn40Pb58Sb2（HL603）锡的质量分数为 40％，是应用最广的锡铅钎料。其熔点低，铺展性好，可得到较光洁的表面，并且力学性能也较好。常用于钎焊铜及铜合金、钢、锌制零件，如钎焊散热器、无线电及电气开关设备、

{'input': '什么是钎料，国内市场占有率是多少?',
 'output': '钎料是一种用于钎焊的金属材料，它能够与母材发生作用并形成牢固的结合。根据搜索结果，我没有找到确切的国内市场占有率数据。不过，全球钎料市场的主要企业包括Harris、铟泰公司、Senju Metal Industry等，它们在国际和国内市场的占有率数据可以通过购买相关市场研究报告来获取。如果需要具体的国内市场占有率，建议进行更深入的市场调研或咨询专业机构。'}

In [42]:

from langchain_core.prompts import PromptTemplate

template = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''

prompt = PromptTemplate.from_template(template)

In [48]:
# Import relevant functionality
from langchain_anthropic import ChatAnthropic
# from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
# from langgraph.checkpoint.memory import MemorySaver
# from langgraph.prebuilt import create_react_agent
# memory = MemorySaver()
model = ChatAnthropic(model_name="claude-3-sonnet-20240229")
search = TavilySearchResults(max_results=2)
tools = [search]
prompt = hub.pull("hwchase17/react")
agent_executor = create_react_agent(model, tools, prompt=prompt)

# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]}, config
):
    print(chunk)
    print("----")

for chunk in agent_executor(
    {"messages": [HumanMessage(content="whats the weather where I live?")]}, config
):
    print(chunk)
    print("----")

c:\Users\admin\anaconda3\envs\qianwen\lib\site-packages\langsmith\client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


TypeError: 'RunnableSequence' object is not callable